### Import bibliotek

In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import os

### Wysyłanie żądania dostępu do strony z opiniami o produkcie

In [ ]:


with open("./headers.json", "r") as file:
    headers = json.load(file)


In [ ]:
product_id = '84514582'
next_page = f'https://www.ceneo.pl/{product_id}#tab=reviews'
all_reviews = []
while next_page:
    res = requests.get(next_page, headers=headers)
    print(res.status_code)
    print(next_page)
    if(res.status_code == 200):
    
        page_dom = BeautifulSoup(res.text, "html.parser")
        reviews = page_dom.select('div.js_product-review:not(.user-post--highlight)')
        
        
        for review in reviews:
            try:
                single_review = {
                    "review_id":review.attrs['data-entry-id'],
                    "author": review.select_one('.user-post__author-name').text.strip(),
                    "content": review.select_one('.user-post__text').text.strip(),
                    "score": review.select_one('.user-post__score .user-post__score-count').text.strip(),
                    "recomendation": review.select_one('.user-post__author-recomendation > em').text.strip() ,
                    "pros":[item.text.strip() for item in review.select('.review-feature__item.review-feature__item--positive')],
                    "cons":[item.text.strip() for item in review.select('.review-feature__item.review-feature__item--negative')],
                    "likes": review.select_one('.vote-yes > span').text.strip(),
                    "dislikes": review.select_one('.vote-no > span').text.strip(),
                    "publish_date": review.select_one(".user-post__published time:nth-of-type(1)")['datetime'].strip(),
                    "purchase_date": review.select_one(".user-post__published time:nth-of-type(2)")['datetime'].strip(),
                
                    }
                all_reviews.append(single_review)
            except (AttributeError, TypeError):
                pass

        print(all_reviews)

        try:
            next_page = 'https://www.ceneo.pl' + page_dom.select_one('.pagination__next')['href']
        except TypeError:
            next_page = None
        

### Zapisanie wszystkich opinii o konkretnym produkcie w bazie danych


In [ ]:
if not os.path.exists('./opinions'):
    os.mkdir('./opinions')

In [ ]:
with open(f'./opinions/{product_id}.json', 'w', encoding="UTF-8") as file:
    json.dump(all_reviews, file, indent=4,ensure_ascii=False)